# Generate a Hexgrid Geodataframe for Los Angeles County

In [ ]:
import geopandas  as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import libpysal
from tobler.util import h3fy
from h3 import h3
#import descartes


import os
from os.path import isfile, join
from pathlib import Path
from os import listdir
import os
os.getcwd()

path =  Path(os.getcwd())
root = path.parent.absolute()

# import libraries needed for upload / download to AWS
import boto3
import awswrangler
from fiona.session import AWSSession
import fiona
# set name of s3 bucket
s3_bucket = 'traffic-data-bucket'

#### Troubleshooting AWS Wrangler
If you cannot import AWS Wrangler due to the error `AttributeError: module 'multiprocessing' has no attribute 'connection'` then try downgrading some of the dependencies by using: `pip install fsspec==0.6.3 PyAthena==1.10.2 s3fs==0.4.0`

## 1. Create Boto3 session
Start by creating a boto3 session so that we can connect to the S3 bucket.

In [ ]:
from aws_secrets import aws_access_key_id, aws_secret_access_key, aws_session_token

my_session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_session_token = aws_session_token

)

## 2. Read shapefile of Los Angeles County from S3 bucket

Next, use fiona to read the shapefile into GeoPandas.  If you try and read the shapefile from the S3 bucket directly into GeoPandas without using Fiona it will issue a timeout error.  The link below explains in further detail.

https://github.com/Toblerity/Fiona/issues/1055

In [ ]:
with fiona.Env(session=AWSSession(my_session)):
    gdf = gpd.read_file(f"s3://{s3_bucket}/raw_data/la_county_website_data/County_Boundaries/County_Boundaries.shp")

## 3. Create a hexgrid over Los Angeles County

### what does `.eq(22)` mean???

In [ ]:
gdf = gdf.loc[gdf['OBJECTID'].eq(22)]

In [ ]:
type(gdf)

In [ ]:
# set the hexagon level
h3_level = 8

In [ ]:
gdf_hex = h3fy(source=gdf, resolution=h3_level)

In [ ]:
print(gdf_hex.columns)
gdf_hex.reset_index(inplace = True)
gdf_hex.shape

In [ ]:
gdf_hex.sample(3)

## 4. Visualize hexgrids over Los Angeles County

In [ ]:
import contextily as ctx
ax = gdf_hex.plot(markersize=0.01, alpha = .3, edgecolors= "black", figsize=(20, 25))

ctx.add_basemap(ax)
plt.show()

## 5. Create shapefile of hexgrids

In [ ]:
gdf_hex[['hex_id', 'geometry']].to_file((root / 'X.data' / 'h3_processed_data' / 'base_map_hex_all' / 'base_map_hex_all.shp'))

### 5.1 Create Boto3 session and upload shapefile data to S3 bucket

In [ ]:
from aws_secrets import aws_access_key_id, aws_secret_access_key, aws_session_token

my_session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_session_token = aws_session_token

)

# create client session
s3 = my_session.client('s3')

In [ ]:
basemap_data_path = root / 'X.data' / 'h3_processed_data' / 'base_map_hex_all' 

for file in os.listdir(basemap_data_path):
    upload_file_key = 'h3_processed_data/base_map_hex_all/' + str(file)
    local_file_path = '../X.data/h3_processed_data/base_map_hex_all/' + str(file)
    # upload to s3
    s3.upload_file(Filename = local_file_path, Bucket = s3_bucket, Key = upload_file_key)